# Test code for the test metrics

In [2]:
import sys
import os
sys.path.append(os.path.abspath("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/topological-metrics-kaggle/src"))


import topometrics.leaderboard

In [3]:
"""
Vesuvius competition metric.

Expects standard Kaggle paths and Linux in order to manage dependencies.
"""

import glob
import importlib
import os
import subprocess
import sys
import numpy as np
import pandas as pd
from PIL import Image, ImageSequence


class ParticipantVisibleError(Exception):
    pass


class HostVisibleError(Exception):
    pass


In [4]:
def load_volume(path):
    im = Image.open(path)
    slices = []
    for i, page in enumerate(ImageSequence.Iterator(im)):
        slice_array = np.array(page)
        slices.append(slice_array)
    volume = np.stack(slices, axis=0)
    return volume


In [5]:
def install_dependencies():
    """On Kaggle, the topometrics library must be installed during the run. This function handles the entire process."""
    try:
        import topometrics.leaderboard

        return None
    # The broad exception is necessary as the initial import can fail for multiple reasons.
    except:
        pass

    resources_dir = '/kaggle/input/vesuvius-metric-resources'
    install_dir = '/kaggle/working/topological-metrics-kaggle'

    try:
        subprocess.run(
            f'cd {resources_dir} && uv pip install --no-index --find-links=wheels -r topological-metrics-kaggle/requirements.txt',
            shell=True,
            check=True,
        )
        subprocess.run(f'cd /kaggle/working && cp -r {resources_dir}/topological-metrics-kaggle .', shell=True, check=True)
        subprocess.run(
            f'cd {install_dir} && chmod +x scripts/setup_submodules.sh scripts/build_betti.sh && make build-betti',
            shell=True,
            check=True,
        )
        subprocess.run(
            f'cd {install_dir} && uv pip install -e . --no-deps --no-index --no-build-isolation -v',
            shell=True,
            check=True,
        )
        # Add the new library to Python's path and invalidate caches to ensure it's found.
        sys.path.append('/kaggle/working/topological-metrics-kaggle/src')
        importlib.invalidate_caches()

    except Exception as err:
        raise HostVisibleError(f'Failed to install topometrics library: {err}')


In [6]:
import os
import glob
import pandas as pd

def generate_standard_submission(submission_dir: str) -> None:
    # 1. Look for TIFs in the provided submission directory
    submission_tifs = glob.glob(os.path.join(submission_dir, '**/*.tif'), recursive=True)
    
    # 2. Local fallback: if not found, look in a local 'tmp' folder instead of /kaggle/tmp
    if len(submission_tifs) == 0:
        submission_tifs = glob.glob('tmp/**/*.tif', recursive=True)
        
    if len(submission_tifs) == 0:
        # Using a standard ValueError for local debugging
        raise ValueError(f'No submission files found in {submission_dir} or local tmp folder.')

    df = pd.DataFrame({'tif_paths': submission_tifs})
    
    # Use os.path.basename for cross-platform compatibility (Windows/Linux)
    df['id'] = df['tif_paths'].apply(lambda x: os.path.basename(x).split('.')[0])

    # 3. Save to the current working directory or a specific output path
    # Removed: os.chdir('/kaggle/working') 
    
    output_path = 'submission.csv'
    df[['id', 'tif_paths']].to_csv(output_path, index=False)
    print(f"Submission saved to {os.path.abspath(output_path)}")

In [13]:
def score_single_tif(
    gt_path,
    pred_path,
    surface_tolerance,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
):
    gt: np.ndarray = load_volume(gt_path)
    pr: np.ndarray = load_volume(pred_path)

    #install_dependencies()
    # The import is here to ensure dependencies are loaded first.
    try:
        # Use a standard import now that the path is reliably set.
        import topometrics.leaderboard
    except Exception as err:
        raise HostVisibleError(f'Failed to import topometrics after installation: {err}')

    score_report = topometrics.leaderboard.compute_leaderboard_score(
        predictions=pr,
        labels=gt,
        dims=(0, 1, 2),
        spacing=(1.0, 1.0, 1.0),  # (z, y, x)
        surface_tolerance=surface_tolerance,  # in spacing units
        voi_connectivity=voi_connectivity,
        voi_transform=voi_transform,
        voi_alpha=voi_alpha,
        combine_weights=(topo_weight, surface_dice_weight, voi_weight),  # (Topo, SurfaceDice, VOI)
        fg_threshold=None,  # None => legacy "!= 0"; else uses "x > threshold"
        ignore_label=2,  # voxels with this GT label are ignored
        ignore_mask=None,  # or pass an explicit boolean mask
    )
    
    return {
        'image_score': np.clip(score_report.score, 0.0, 1.0),
        'topo_score': score_report.topo.toposcore,
        'surface_dice': score_report.surface_dice,
        'voi_score': score_report.voi.voi_score,
        'voi_split': score_report.voi.voi_split,
        'voi_merge': score_report.voi.voi_merge
    }

In [ ]:
def score(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    row_id_column_name: str,
    surface_tolerance: float = 2.0,
    voi_connectivity: int = 26,
    voi_transform: str = 'one_over_one_plus',
    voi_alpha: float = 0.3,
    topo_weight: float = 0.3,
    surface_dice_weight: float = 0.35,
    voi_weight: float = 0.35,
    ) -> float:
    """Returns the mean per-volume scores and attaches sub-metrics to the solution df."""
    if not solution['tif_paths'].apply(os.path.exists).all():
        raise HostVisibleError('Invalid solution file paths')

    solution['pred_paths'] = submission['tif_paths']
    
    # Apply the scoring function and expand the returned dict into new columns
    metrics_df = solution.apply(
        lambda row: score_single_tif(
            row['tif_paths'],
            row['pred_paths'],
            surface_tolerance,
            voi_connectivity=voi_connectivity,
            voi_transform=voi_transform,
            voi_alpha=voi_alpha,
            topo_weight=topo_weight,
            surface_dice_weight=surface_dice_weight,
            voi_weight=voi_weight,
        ),
        axis=1,
    ).apply(pd.Series)

    # Merge the new metric columns back into the original solution dataframe
    solution = pd.concat([solution, metrics_df], axis=1)

    # Return the mean of the primary leaderboard score
    return float(np.mean(solution['image_score'])), solution

#### small tests

In [19]:
score_report = score_single_tif(
    gt_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_labels/3925240194.tif",
    pred_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_pred/3925240194.tif",
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(f"score_report: {score_report}")

score_report: {'image_score': np.float64(0.3493847817704987), 'topo_score': 0.007314644154704989, 'surface_dice': 0.5816345181502148, 'voi_score': 0.41033802049003426, 'voi_split': 2.676790371813977, 'voi_merge': 2.1132603511304806}


In [ ]:
score_report: 0.2753458417894591

In [21]:
# Print summary of results
print("-" * 30)
print(f"Leaderboard score: {score_report['image_score']:.4f}")
print(f"Topo score:        {score_report['topo_score']:.4f}")
print(f"Surface Dice:      {score_report['surface_dice']:.4f}")
print(f"VOI score:         {score_report['voi_score']:.4f}")
print(f"VOI split/merge:   {score_report['voi_split']:.4f}, {score_report['voi_merge']:.4f}")
print("-" * 30)

------------------------------
Leaderboard score: 0.3494
Topo score:        0.0073
Surface Dice:      0.5816
VOI score:         0.4103
VOI split/merge:   2.6768, 2.1133
------------------------------


## Folder test!

In [29]:
import pandas as pd
import numpy as np
import os

# --- Constants & Paths ---
solution_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_labels/test_labels_df.csv"
submission_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_pred/test_pred_df.csv"
output_file = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_pred/detailed_scores.csv"

# 1. Load the solution and submission dataframes
solution = pd.read_csv(solution_path)
submission = pd.read_csv(submission_path)

# 2. Call the score function
# Note: 'row_id_column_name' should match the ID column in your CSV (e.g., 'id' or 'segment_id')
final_score, solution = score(
    solution=solution,
    submission=submission,
    row_id_column_name='id'
)

# 3. Save to disk so you don't lose the hard work!

solution.to_csv(output_file, index=False)
print(f"\nFinal Mean Score: {final_score}")
print(f"Saved detailed row-by-row metrics to: {output_file}")


Final Mean Score: 0.3123653117799789
Saved detailed row-by-row metrics to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/detailed_scores.csv


In [30]:
# test using object-oriented approach
import sys
import os
sys.path.append(os.path.abspath("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/utils"))
from test_obj import VesuviusMetric

test_metric_obj = VesuviusMetric(
    solution_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_labels/test_labels_df.csv",
    submission_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_pred/test_pred_df.csv",
    output_file="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_pred/detailed_scores_obj.csv"
)
test_metric_obj._run()


Final Mean Score: 0.3123653117799789
Saved detailed row-by-row metrics to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_pred/detailed_scores_obj.csv
